In [97]:
import json
import numpy as np
import pandas as pd
from typing import Any, Dict, List
from mapping_functions import get_mapping_for_dict, flatten_series_by_mapping

In [98]:
with open('jobs.json','r') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data['jobs'])
drop_cols = ['account', 'cluster', 'container', 'comment', 'array','association', 'derived_exit_code', 'exit_code', 'het', 'job_id', 'name', 'mcs', 'kill_request_user']
drop_rows_indices = [8679]
df = df.drop(columns=drop_cols)
df = df.drop(index=drop_rows_indices).reset_index()
df.head()

,index,allocation_nodes,constraints,time,flags,group,nodes,partition,priority,qos,required,reservation,state,steps,tres,user,wckey,working_directory
0,0,1,None,"{'elapsed': 9482, 'eligible': 1672509600, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1788,standard,1417,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1788'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
1,1,1,None,"{'elapsed': 9932, 'eligible': 1672531200, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e2281,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e2281'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
2,2,1,None,"{'elapsed': 10601, 'eligible': 1672552800, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1901,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1901'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
3,3,1,None,"{'elapsed': 9408, 'eligible': 1672574400, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1403,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1403'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
4,4,1,None,"{'elapsed': 10631, 'eligible': 1672617600, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1893,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1893'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12823 entries, 0 to 12822
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              12823 non-null  int64 
 1   allocation_nodes   12823 non-null  int64 
 2   constraints        116 non-null    object
 3   time               12823 non-null  object
 4   flags              12823 non-null  object
 5   group              12823 non-null  object
 6   nodes              12823 non-null  object
 7   partition          12823 non-null  object
 8   priority           12823 non-null  int64 
 9   qos                12351 non-null  object
 10  required           12823 non-null  object
 11  reservation        12823 non-null  object
 12  state              12823 non-null  object
 13  steps              12823 non-null  object
 14  tres               12823 non-null  object
 15  user               12823 non-null  object
 16  wckey              12823 non-null  objec

In [135]:
# 
def fix_steps_tres_allocated_order(row):
    '''Fix order in steps-tres-allocated where node and energy are in the wrong positions'''
    ret = row[0]['tres']['allocated'][0:2]

    if row[0]['tres']['allocated'][2]['type'] == 'energy' and row[0]['tres']['allocated'][3]['type'] == 'node':
        ret += [row[0]['tres']['allocated'][3]]
    else:
        ret += [row[0]['tres']['allocated'][2]]
    row[0]['tres']['allocated'] = ret
    return row

# Fix order in tres-allocated
def fix_tres_allocated_order(row):
    '''Fix order in tres-allocated where node and energy are in the wrong positions'''
    ret = row['allocated'][0:2]

    if row['allocated'][2]['type'] == 'energy' and row['allocated'][3]['type'] == 'node':
        ret += [row['allocated'][3]]
    else:
        ret += [row['allocated'][2]]
    row['allocated'] = ret
    return row

df['steps'] = df['steps'].apply(fix_steps_tres_allocated_order)
df['tres'] = df['tres'].apply(fix_tres_allocated_order)

df.head(5)

,index,allocation_nodes,constraints,time,flags,group,nodes,partition,priority,qos,required,reservation,state,steps,tres,user,wckey,working_directory
0,0,1,None,"{'elapsed': 9482, 'eligible': 1672509600, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1788,standard,1417,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1788'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
1,1,1,None,"{'elapsed': 9932, 'eligible': 1672531200, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e2281,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e2281'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
2,2,1,None,"{'elapsed': 10601, 'eligible': 1672552800, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1901,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1901'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
3,3,1,None,"{'elapsed': 9408, 'eligible': 1672574400, 'end...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1403,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1403'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...
4,4,1,None,"{'elapsed': 10631, 'eligible': 1672617600, 'en...","[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1893,standard,1434,normal,"{'CPUs': 24, 'memory': 2048}","{'id': 0, 'name': 0}","{'current': 'COMPLETED', 'reason': 'BeginTime'}","[{'nodes': {'list': ['e1893'], 'count': 1, 'ra...","{'allocated': [{'type': 'cpu', 'name': None, '...",kulka,"{'wckey': '', 'flags': []}",/mnt/storage_2/scratch/grant_68/kulka/test/dev...


Transform steps

In [136]:
steps_series = df['steps'].apply(lambda x: x[0] if type(x) is list and len(x) > 0 else 'lol')
steps_mapping_list = get_mapping_for_dict(steps_series.iloc[0], 'steps')
steps_df = flatten_series_by_mapping(steps_series, steps_mapping_list)
steps_df

,steps-nodes-list,steps-nodes-count,steps-nodes-range,steps-tres-requested-max-#0-type,steps-tres-requested-max-#0-name,steps-tres-requested-max-#0-id,steps-tres-requested-max-#0-count,steps-tres-requested-max-#0-task,steps-tres-requested-max-#0-node,steps-tres-requested-max-#1-type,...,steps-CPU-governor,steps-kill_request_user,steps-state,steps-statistics-CPU-actual_frequency,steps-statistics-energy-consumed,steps-step-job_id,steps-step-het-component,steps-step-id,steps-step-name,steps-task-distribution
0,[e1788],1,e1788,cpu,None,1,150077410,0,e1788,mem,...,[],None,COMPLETED,0,0,25977766,None,batch,batch,Unknown
1,[e2281],1,e2281,cpu,None,1,155892610,0,e2281,mem,...,[],None,COMPLETED,0,0,26081263,None,batch,batch,Unknown
2,[e1901],1,e1901,cpu,None,1,180730980,0,e1901,mem,...,[],None,COMPLETED,0,0,26081264,None,batch,batch,Unknown
3,[e1403],1,e1403,cpu,None,1,153234590,0,e1403,mem,...,[],None,COMPLETED,0,0,26081265,None,batch,batch,Unknown
4,[e1893],1,e1893,cpu,None,1,181754220,0,e1893,mem,...,[],None,COMPLETED,0,0,26081267,None,batch,batch,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12818,[e1427],1,e1427,cpu,None,1,510,0,e1427,mem,...,[],None,COMPLETED,0,575766,27427892,None,batch,batch,Unknown
12819,[e1329],1,e1329,cpu,None,1,480,0,e1329,mem,...,[],None,COMPLETED,0,177408,27428386,None,batch,batch,Unknown
12820,[e1427],1,e1427,cpu,None,1,3663190,0,e1427,mem,...,[],None,COMPLETED,0,48800,27428399,None,batch,batch,Unknown
12821,[e1427],1,e1427,cpu,None,1,75308460,0,e1427,mem,...,[],None,COMPLETED,0,1384508,27428511,None,batch,batch,Unknown


Transform TRES column

In [137]:
tres_series = df['tres']
tres_mapping_list = get_mapping_for_dict(tres_series.iloc[0], 'tres')
tres_df = flatten_series_by_mapping(tres_series, tres_mapping_list)
tres_df

,tres-allocated-#0-type,tres-allocated-#0-name,tres-allocated-#0-id,tres-allocated-#0-count,tres-allocated-#1-type,tres-allocated-#1-name,tres-allocated-#1-id,tres-allocated-#1-count,tres-allocated-#2-type,tres-allocated-#2-name,...,tres-requested-#1-id,tres-requested-#1-count,tres-requested-#2-type,tres-requested-#2-name,tres-requested-#2-id,tres-requested-#2-count,tres-requested-#3-type,tres-requested-#3-name,tres-requested-#3-id,tres-requested-#3-count
0,cpu,None,1,24,mem,None,2,49152,node,None,...,2,49152,node,None,4,1,billing,None,5,24
1,cpu,None,1,24,mem,None,2,49152,node,None,...,2,49152,node,None,4,1,billing,None,5,24
2,cpu,None,1,24,mem,None,2,49152,node,None,...,2,49152,node,None,4,1,billing,None,5,24
3,cpu,None,1,24,mem,None,2,49152,node,None,...,2,49152,node,None,4,1,billing,None,5,24
4,cpu,None,1,24,mem,None,2,49152,node,None,...,2,49152,node,None,4,1,billing,None,5,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12818,cpu,None,1,96,mem,None,2,327680,node,None,...,2,327680,node,None,4,4,billing,None,5,96
12819,cpu,None,1,96,mem,None,2,327680,node,None,...,2,327680,node,None,4,4,billing,None,5,96
12820,cpu,None,1,96,mem,None,2,327680,node,None,...,2,327680,node,None,4,4,billing,None,5,96
12821,cpu,None,1,96,mem,None,2,327680,node,None,...,2,327680,node,None,4,4,billing,None,5,96


Transform time column

In [138]:
time_series = df['time']
time_mapping_list = get_mapping_for_dict(time_series.iloc[0], 'time')
time_df = flatten_series_by_mapping(time_series, time_mapping_list)
time_df

,time-elapsed,time-eligible,time-end,time-start,time-submission,time-suspended,time-system-seconds,time-system-microseconds,time-limit,time-total-seconds,time-total-microseconds,time-user-seconds,time-user-microseconds
0,9482,1672509600,1672529962,1672509600,1669893940,0,0,0,1380,0,0,0,0
1,9932,1672531200,1672555653,1672531200,1671039773,0,0,0,1380,0,0,0,0
2,10601,1672552800,1672595012,1672552800,1671039773,0,0,0,1380,0,0,0,0
3,9408,1672574400,1672604330,1672574400,1671039773,0,0,0,1380,0,0,0,0
4,10631,1672617600,1672629296,1672617600,1671039774,0,0,0,1380,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12818,1150,1674827652,1674828802,1674827652,1674827652,0,0,0,2880,0,0,0,0
12819,549,1674828526,1674829078,1674828526,1674828526,0,0,0,2880,0,0,0,0
12820,137,1674829182,1674829319,1674829182,1674829182,0,0,0,2880,0,0,0,0
12821,2695,1674829551,1674832246,1674829551,1674829551,0,0,0,2880,0,0,0,0


Transform required

In [139]:
required_series = df['required']
required_mapping_lsit = get_mapping_for_dict(required_series.iloc[0], 'required')
required_df = flatten_series_by_mapping(required_series, required_mapping_lsit)
required_df

,required-CPUs,required-memory
0,24,2048
1,24,2048
2,24,2048
3,24,2048
4,24,2048
...,...,...
12818,96,81920
12819,96,81920
12820,96,81920
12821,96,81920


Transform state

In [140]:
state_series = df['state']
state_mapping_List = get_mapping_for_dict(state_series.iloc[0], 'state')
state_df = flatten_series_by_mapping(state_series, state_mapping_List)
state_df

,state-current,state-reason
0,COMPLETED,BeginTime
1,COMPLETED,BeginTime
2,COMPLETED,BeginTime
3,COMPLETED,BeginTime
4,COMPLETED,BeginTime
...,...,...
12818,COMPLETED,None
12819,COMPLETED,None
12820,COMPLETED,None
12821,COMPLETED,None


Transform wckey

In [141]:
wckey_series = df['wckey']
wckey_mapping_List = get_mapping_for_dict(wckey_series.iloc[0], 'wckey')
wckey_df = flatten_series_by_mapping(wckey_series, wckey_mapping_List)
wckey_df

,wckey-wckey,wckey-flags
0,,[]
1,,[]
2,,[]
3,,[]
4,,[]
...,...,...
12818,,[]
12819,,[]
12820,,[]
12821,,[]


MERGE ALL DATAFRAMES

In [166]:
to_be_merged = ['steps', 'tres', 'state', 'time', 'required', 'wckey']

# Drop columns that were flattened
df_flat = df.drop(columns=to_be_merged)

# Merge all dataframes
df_flat = pd.concat([df_flat, required_df, state_df, time_df, wckey_df, tres_df, steps_df], axis='columns')

assert df_flat.shape[0] == df.shape[0], 'Shapes of DataFrame before and after merge should be the same!'

df_flat.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12823 entries, 0 to 12822
Columns: 260 entries, index to steps-task-distribution
dtypes: int64(136), object(124)
memory usage: 25.4+ MB


In [167]:
df_flat.to_csv('jobs_flattened.csv', index=False)

In [177]:
# list(filter(lambda x: x is not None, [x if 'tres-allocated-' in x else None for x in df_flat.columns]))

In [176]:
def merge_type_and_counts(column_core: str) -> pd.DataFrame:
    '''
    Merge types and counts into one column where colname 
    is of specific type and values are corresponding counts
    '''    
    unique_values = df_flat[f'{column_core}-type'].unique()
    name_after = f'{column_core}-{unique_values[0]}'

    if unique_values.shape[0] == 1:
        counts = df_flat[f'{column_core}-count'].to_numpy()
    else:
        print(f'More than one value. Be careful: {column_core}-type, {unique_values}')

    #print(df_flat[f'{column_core}-{unique_values[0]}'].value_counts())
    return pd.DataFrame(data=counts, columns=[name_after])

def merge_all_tres_possible(df: pd.DataFrame) -> pd.DataFrame:
    '''Extract types and counts from tres fields and use them as new columns'''
    dataframes_to_merge = []
    columns_to_drop = []

    column_cores_to_go_over = []

    # Add requested and consumed paths as they share the same colpath type
    for what in ['requested', 'consumed']:
        for aggregation_type in ['max', 'min', 'average', 'total']:
            for i in range(6):
                to_merge = f'steps-tres-{what}-{aggregation_type}-#{i}'
                if f'{to_merge}-type' in df_flat.columns:
                    column_cores_to_go_over.append(to_merge)
                else:
                    print(f'Colpath does not exist! : {to_merge}-type')

    # Add allocated paths
    for i in range(6):
        to_merge = f'steps-tres-allocated-#{i}'
        if f'{to_merge}-type' in df_flat.columns:
            column_cores_to_go_over.append(to_merge)
        else:
            print(f'Colpath does not exist! : {to_merge}-type')

    # Add requested and consumed paths as they share the same colpath type
    for what in ['allocated', 'requested']:
        for i in range(6):
            to_merge = f'tres-{what}-#{i}'
            if f'{to_merge}-type' in df_flat.columns:
                column_cores_to_go_over.append(to_merge)
            else:
                print(f'Colpath does not exist! : {to_merge}-type')

    for col in column_cores_to_go_over:
        coldf = merge_type_and_counts(col)
        columns_to_drop.append(f'{col}-type')
        columns_to_drop.append(f'{col}-name')
        columns_to_drop.append(f'{col}-id')
        columns_to_drop.append(f'{col}-count')
        dataframes_to_merge.append(coldf)

    df = pd.concat([df] + dataframes_to_merge, axis='columns')
    df = df.drop(columns=columns_to_drop)

    print(df.columns.to_list())

    return df 

def remove_index_element_from_column_names(_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Remove "#id" part from column names. 
    Should be used only after flattening and merging type-counts operations.
    '''
    rename_map = {}
    for old in _df.columns.to_list():
        new = '-'.join(filter(lambda x: '#' not in x, old.split('-')))
        rename_map[old] = new

    _df = _df.rename(columns=rename_map)
    return _df
    
df_flat_merged = merge_all_tres_possible(df_flat)   

df_flat_merged = remove_index_element_from_column_names(df_flat_merged)

df_flat_merged.to_csv('jobs_flattened_cleaned.csv', index=False)
df_flat_merged.head(5) 

Colpath does not exist! : steps-tres-consumed-max-#2-type
Colpath does not exist! : steps-tres-consumed-max-#3-type
Colpath does not exist! : steps-tres-consumed-max-#4-type
Colpath does not exist! : steps-tres-consumed-max-#5-type
Colpath does not exist! : steps-tres-consumed-min-#2-type
Colpath does not exist! : steps-tres-consumed-min-#3-type
Colpath does not exist! : steps-tres-consumed-min-#4-type
Colpath does not exist! : steps-tres-consumed-min-#5-type
Colpath does not exist! : steps-tres-consumed-average-#2-type
Colpath does not exist! : steps-tres-consumed-average-#3-type
Colpath does not exist! : steps-tres-consumed-average-#4-type
Colpath does not exist! : steps-tres-consumed-average-#5-type
Colpath does not exist! : steps-tres-consumed-total-#2-type
Colpath does not exist! : steps-tres-consumed-total-#3-type
Colpath does not exist! : steps-tres-consumed-total-#4-type
Colpath does not exist! : steps-tres-consumed-total-#5-type
Colpath does not exist! : steps-tres-allocated-#

,index,allocation_nodes,constraints,flags,group,nodes,partition,priority,qos,reservation,...,steps-tres-allocated-cpu,steps-tres-allocated-mem,steps-tres-allocated-node,tres-allocated-cpu,tres-allocated-mem,tres-allocated-node,tres-requested-cpu,tres-requested-mem,tres-requested-node,tres-requested-billing
0,0,1,None,"[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1788,standard,1417,normal,"{'id': 0, 'name': 0}",...,24,49152,1,24,49152,1,24,49152,1,24
1,1,1,None,"[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e2281,standard,1434,normal,"{'id': 0, 'name': 0}",...,24,49152,1,24,49152,1,24,49152,1,24
2,2,1,None,"[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1901,standard,1434,normal,"{'id': 0, 'name': 0}",...,24,49152,1,24,49152,1,24,49152,1,24
3,3,1,None,"[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1403,standard,1434,normal,"{'id': 0, 'name': 0}",...,24,49152,1,24,49152,1,24,49152,1,24
4,4,1,None,"[CLEAR_SCHEDULING, STARTED_ON_SCHEDULE]",staff,e1893,standard,1434,normal,"{'id': 0, 'name': 0}",...,24,49152,1,24,49152,1,24,49152,1,24
